# Warning
This file is currently a work in progress. Expect instability, inconsistency, and ugly code.
@W

In [445]:
DATA_DIRECTORY = "../../data"

# Delete me, eventually
data_path = DATA_DIRECTORY + '/dialog-babi-task1/'
training_file = data_path + 'dialog-babi-task1-API-calls-trn-workspace.txt'
dev_file = data_path + 'dialog-babi-task1-API-calls-dev-workspace.txt'

In [463]:
from functional import seq

In [447]:
# Initial object constructors
class Workspace():
    def __init__(self, tree):
        # assert tree is array
        self.tree = tree
        
    def add(address, content):
        assert isinstance(address, (str))
        assert isinstance(content, (str))
        
        def _add(tree):
            if not isinstance(tree, (list)):
                return tree
            elif tree[0] == address:
                return tree.concat([[content]])
            else:
                return map(tree, _add)
        
        return Workspace(_add(self.tree))

    def update(self, action):
        if action.what == 'INIT':
            return Workspace(['root'])
        elif action.what == 'MSG':
            return self
        elif action.what == 'ADD':
            tmp = action.content.split(' ')
            assert (len(tmp) == 2)
            address = tmp[0]
            address = tmp[1]
            return this.add(address, content)
        
    # TODO: Figure out how to flattenDeep    
    def to_words():
        return "NOT IMPLEMENTED YET"
    def to_tree():
        return this.tree
        

In [448]:
class Action():
    def __init__(self, who, what, content=False):
        self.who = who
        self.what = what
        self.content = content
        
    def to_words(self):
        return [f"{self.who}:", self.what] + (self.content.split(' ') if self.content else [])
        
    def to_string(self):
        if self.content:
            return f'{self.who}: {self.what} {self.content}'
        else:
            return f'{self.who}: {self.what}'

In [449]:
def files_to_words(filenames):
     return (seq(filenames)
                .map(lambda filename: open(filename).read().split())
                .flatten()
                .distinct()
                .sorted()
                .to_list())

In [450]:
def string_to_action(s):
    who = s[0]
    i = s.index(" ")
    line = s[i+1:]
    j = line.index(" ")
    what = line[0:j]
    content = line[j+1:]
    return Action(who, what, content)
    

In [464]:
def file_to_dialogs(filename):
    init_entry = {
        "workspace": Workspace([]),
        "action": Action(who='A', what='INIT')
    }
    
    action_list = [init_entry]
    current_workspace = init_entry["workspace"]
    
    for line in open(filename, 'r'):
        line = line.split('\n')
        for dialog in line:
            #line = line[:-1]
            if not dialog:
                continue
            try: 
                action_list.append({"workspace": current_workspace,
                                    "action": string_to_action(dialog)})
            except ValueError:
                print("This shouldn't happen!")
                return

    return action_list
   

In [465]:
for line in open(training_file, 'r'):
    line = line.split('\n')
    for dialog in line:
        print(dialog)
    break

A: ADD root cuisine



In [466]:
special_words = [
  '^', '$',            # start/end markers
  'INIT', 'WAIT',      # action types not in data
  'NO_OUTPUT',         # filler when copy or match word is not available
  'NO_USER_ACTION',    # initial value of lastUserAction
  'NO_WORKSPACE_WORD'
]

In [467]:
def load_babi_data():
    data_path = DATA_DIRECTORY + '/dialog-babi-task1/'
    training_file = data_path + 'dialog-babi-task1-API-calls-trn-workspace.txt'
    dev_file = data_path + 'dialog-babi-task1-API-calls-dev-workspace.txt'
    types_and_tokens = [
            { "type": 'cuisine', "tokens": ['french', 'italian', 'british', 'spanish', 'indian'] },
            { "type": 'location', "tokens": ['rome', 'london', 'bombay', 'paris', 'madrid'] },
            { "type": 'price', "tokens": ['cheap', 'moderate', 'expensive'] },
            { "type": 'people', "tokens": ['two', 'four', 'six', 'eight'] },
        ]
    
    data = {
      "training": file_to_dialogs(training_file),
      "dev": file_to_dialogs(dev_file),
      "words": files_to_words([training_file]) + special_words,
      "types_and_tokens": types_and_tokens
    }

    return data

In [468]:
def create_one_hot(vocab_size):
    return ([[int(i == j) for i in range(vocab_size)] for j in range(vocab_size)])


In [469]:
vocab_size = len(data["words"])
one_hot = create_one_hot(vocab_size)

In [470]:
def action_to_one_hot(action):
    return ((seq(['^'] + action.to_words() + ['$'])
                .map(lambda w: data['words'].index(w))
                .map(lambda i: one_hot[i])
                .to_list()))

In [471]:
MAX_TENSOR_SIZE = 23 #TODO: Refactor to determine automatically
def pad_tensor(tensor):
    m = MAX_TENSOR_SIZE
    a = tensor
    return a + [([0 for i in range(vocab_size)]) for j in range(m - len(a))]

In [472]:
def create_training_data(data):
    return (seq(data)
                .map(lambda a: a['action'])
                .map(lambda a: action_to_one_hot(a))
                .map(pad_tensor)
                .map(torch.Tensor)
           )

                #.map(action_to_one_hot))

In [473]:
data = load_babi_data()

# Ensure it looks ok
data['training'][100]["action"].to_string()

'A: MSG any preference on a type of cuisine'

In [474]:
# Convert our data into a list of tensors

out = torch.stack(list(create_training_data(data['training'])))
training_data = torch.stack(out)

In [475]:
files_to_dialogs(training_file)

NameError: name 'files_to_dialogs' is not defined

# WARNING!
Everything below this cell is currently non-functional. If it compiles, I'll be rather surprised.

In [34]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [35]:
# Adapted from:
# http://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
# TODO: Implement make OneHot
class Encoder(nn.Module):
    def __init__(self, x_size, h_size, n_layers=1):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.h_size = h_size
        self.cell = nn.GRU(x_size, h_size)

    # x: input
    # h: hidden state
    def forward(self, x, h):
        for i in range(self.n_layers):
            output, h = self.cell(output, h)
        return output, h
    
    def init_hidden(self):
        return Variable(torch.zeros(1, 1, self.h_size)) 

In [36]:
class DecoderMock(nn.Module):
    def __init__(self, x_size, h_size, n_layers=1):
        super(DecoderMock, self).__init__()
        self.n_layers = n_layers
        self.h_size = h_size
       #self.cell = nn.GRU(x_size, h_size)

    # x: input
    # h: hidden state
    def forward(self):
        for i in range(self.n_layers):
            output, h = self.cell(output, h)
        return output, h
    
    def init_hidden(self):
        return Variable(torch.zeros(1, 1, self.h_size)) 

In [37]:
#list(Encoder(10, 10).parameters()

In [38]:
# TODO: Implement initialization


# Whether this is a good idea is still in question
attentional_net = torch.nn.Sequential(
                    torch.nn.Linear(2, 1),
                    torch.nn.Tanh())

# Whether this is a good idea is still in question
decoder_output_net = torch.nn.Sequential(
                        torch.nn.Softmax())

class Decoder(nn.Module):
    def __init__(self, x_size, h_size, n_layers=1):
        self.n_layers = n_layers
        self.h_size = h_size
        self.cell = nn.GRU(x_size, h_size)

        self.out = nn.Linear(h_size, )
        self.attention = torch.nn.Sequential(torch.nn.Linear(2 * x_size, 1),
                                             torch.nn.Tanh())

    def forward(self, decoder_state, encoder_states):
        # Combine State
        combinedState = lambda encoder_state, z: self.attention(torch.cat(decoder_state, encoder_state), 0)

        # This is terrible and I feel dirty
        zero_states = torch.zeros(encoder_states.size()[0])

        # Create attention vector
        attention_vector = T.softmax(torch.Tensor.map_(encoder_states, zero_states,
                                                       combinedState))

        final = encoder_states * torch.transpose(attention_vector.unsqueeze(0), 0, 1).expand(encoder_states.size())

        for i in range(self.n_layers):
            output, h = self.cell(output, h)
        return output, h

In [329]:
def create_one_hot(vocab_size):
    return ([[int(i == j) for i in range(vocab_size)] for j in range(vocab_size)])
